In [14]:
# 引入包
import requests
import pandas as pd
from lxml import etree
from time import sleep
# ,auth=("admin","admin")

In [15]:
class Test_1_Spider():
    def __init__(self):
        #数据初始化
        self.basic_url = 'https://ssr2.scrape.center/'
        self.headers = {
            "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0",
            "accept-encoding":"gzip, deflate, br, zstd",
            "accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
            "accept-language":"zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6",
            "connection":"keep-alive",
            "host":"ssr1.scrape.center",
        }
    
    #构造url，翻页
    def get_url_list(self):
            url_list = [self.basic_url+f"page/{i}" for i in range(1,11)]
            return url_list
    #获取响应
    def get_response(self,url):
        response = requests.get(url,headers=self.headers)
        if response.status_code == 401:
             response = requests.get(url,headers=self.headers,auth=("admin","admin"))
        html_str = response.content.decode()
        return html_str
    #清洗数据
    def get_data(self,html_str):
        HtmlData = etree.HTML(html_str)
        data = pd.DataFrame(columns=["电影名称","类型","产地","时长","得分"])
        div_list = HtmlData.xpath("//div[@class='el-card__body']")
        for div in div_list:
            info_dicts = {}
            info_dicts["电影名称"] = div.xpath(".//a[@class='name']/h2//text()")[0]
            info_dicts["类型"] = "-".join(div.xpath(".//div[@class='categories']//span/text()"))
            info_dicts["产地"] = div.xpath(".//div[@class='m-v-sm info'][1]/span[1]/text()")[0]
            info_dicts["时长"] = div.xpath(".//div[@class='m-v-sm info'][1]/span[3]/text()")[0]
            info_dicts["得分"] = div.xpath(".//p[@class='score m-t-md m-b-n-sm']/text()")[0].replace("\n                ","")
            info_df = pd.DataFrame(info_dicts,index=[0])
            data = pd.concat([data,info_df])

        return data



    def run(self):
         #运行主程序
         #1、获取url列表
        need_data = pd.DataFrame(columns=["电影名称","类型","产地","时长","得分"])
        url_list = self.get_url_list()
        for url in url_list:
            #2、发送请求，获取响应
            html_str = self.get_response(url)
            #3、清洗数据
            data = self.get_data(html_str)
            #保存数据
            need_data = pd.concat([need_data,data])
            sleep(2)
        need_data.to_excel(".\spider数据.xlsx",index=False)

        

In [16]:
if __name__ == "__main__":
    Spider = Test_1_Spider()
    Spider.run()